<img src="images/Carriers.jpg" width="800">

# Inserting external data sources into PostgreSQL - Carriers
Now that we know where to find data about carriers, we can choose between options of how to get this data into Python.

1. Download the csv file directly from the website to your local drive and import it into Python
2. Read the csv directly from the web into Python

Since we are hackers we're obviously going with the second option!

## Importing carrier data from the web into Python
First, set the carrier_columns variable so that it contains the two strings 'carrier' and 'name'.

In [ ]:
# Set column names to 'carrier' and 'name'
carrier_columns = ['carrier' , 'name']

Now think back to last week: We used the Python library Pandas to import dataframes stored on our local drive.   
Find a function in the Pandas package that enables you to read a csv file from the web into python.  
Once you have found the right function import the Pandas library.

In [ ]:
# Import pandas
import pandas as pd

Next, use the Pandas function with the [link](https://raw.githubusercontent.com/dannguyen/bts-transstats-t100-domestic-demo/master/data/lookup-tables/L_UNIQUE_CARRIERS.csv) to the csv file to import the data into Python.

In [ ]:
# Import carriers data from the web
carriers = pd.read_csv('https://raw.githubusercontent.com/dannguyen/bts-transstats-t100-domestic-demo/master/data/lookup-tables/L_UNIQUE_CARRIERS.csv', 
                     names=carrier_columns, # Sets the column names to the values in carrier_columns
                     skiprows = 1) # Skips the row with column names

# Print the carriers dataframe


Congratulations! With just a few lines of code you loaded a csv file from the web into Python and changed the column names on the fly. Good job!

## Inserting carrier data into the database in Python
Now that we have the data in Python, all we have to do is write it into our SQL database. Before we do that we should make sure that our data is clean. Let's run some basic summary statistics before we move on.

In [ ]:
# Print carriers info
carriers.info()

It seems we have one NA/NULL value in the carrier column, since there are 1564 non-null values out of 1565 total values.  
We can confirm this by running the isnull() and sum() function together.

In [ ]:
# Count NULL values
carriers.isna().sum()

Clearly, there is one NA/NULL value in our table. Alright, let's find out which carrier is missing its code value.

In [ ]:
# Find missing carrier code
carriers[carriers.isna().any(axis=1)]

Found it! The North American Airlines is missing their carrier code value. A quick research on the internet reveals that they ceased operations in 2014 and that in cases like that carrier codes can be reassigned to other carriers. This is good to know, but for now we don't have to worry about this missing value and can leave it as is. Let's move on and add the table to our SQL database.

In order to do this, we need to connect to our database, export the data from Python, create a new table in our database and write the data into it.

Before this, please add this code to the end of your sql.py - we'll come back to the "why" later in the notebook:

```python
# sqlalchemy engine for writing data to a database

from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{os.getenv("user")}:{os.getenv("password")}@{os.getenv("host")}:{os.getenv("port")}/{os.getenv("database")}',  
connect_args={'options': '-csearch_path={}'.format(os.getenv("schema"))})
```

Now, import your function to retrieve data from the postgres database from your own sql module:

In [ ]:
# Import get_data function from the sql module
from sql import get_data


Now that we have established a connection, let's check if the connection is working.  
Complete the code below and retrieve the first 5 rows from the flights table.

In [ ]:
# Print top 5 rows of the flights table
get_data('SELECT * FROM flights LIMIT 5')

Next, calculate the total number of rows in the flights table.

In [ ]:
# Total number of rows in flights table
get_data('SELECT count(*) FROM flights LIMIT 5')

Great, everything seems to be working as expected. Next, let's add the carriers data to our database. For this, we are going to use the <ins>to_sql()</ins> function from pandas. While the to_sql() function has multiple arguments, the important ones are

- name: name of SQL table
- con: sqlalchemy.engine or sqlite3.Connection

Feel free to check out the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html) of the to_sql() function.  

We're going to set the parameter name to the name of the table we want to create and for the con argument we can simply use the connection object conn we imported from the sql.py file, right? Well, no.  
Querying data from a database is handled differently than writing to a database. In order to write our carriers data to our PostgreSQL database, we need to create and use a sqlalchemy engine as the connection. Fortunately, creating it is easy and only requires two lines of code. First we need to import the sqlalchemy package and then create the engine with the <ins>create_engine()</ins> function. Earlier in this notebook, you added the function to the sql.py file. Your job now is to complete the code below and import the engine from sql.py.

In [ ]:
# Import engine from sql.py
from sql import engine

# Print engine
engine

Good job! Next, set the table_name variable. Its value will be name of the table that will be written to the PostgreSQL database.

In [ ]:
# IMPORTANT: Set the table_name variable to 'carriers_' + your initials/group number
# Example: carriers_pw for Philipp Wendt or carriers_1 for group 1
table_name = 'table_solution'

Good job! It's time for the final step: writing the carriers data into the database.  
Complete the code below and write carriers data to the database using the to_sql() function.)

In [ ]:
# we need psycopg2 for raising possible error message
import psycopg2

In [ ]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        carriers.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

Did you get 'The carriers table was imported successfully.'? If yes, then you should be able to query the table from the database. Let's do that to make sure we did everything right!

In [ ]:
# Query the newly created table
get_data('SELECT * FROM table_solution LIMIT 1')


It worked, awesome! To summarise, we added an external data source to our PostgreSQL database which extends our existing data and allows us to run even better analyses. What's also great is that we didn't need to write a lot of complicated Python code.  

Congratulations on completing this notebook, you can be proud of yourself!  
Take a break and then move on to the next notebook where you will apply everything you've learned here and extend the existing data even further!